In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import our input dataset
charity_df = pd.read_csv('charity_data.csv')
charity_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
charity_df.shape

(34299, 12)

In [3]:
charity_df.dtypes

EIN                        int64
NAME                      object
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [4]:
# Generate our categorical variable list
charity_cat = charity_df.dtypes[charity_df.dtypes == "object"].index.tolist()


In [5]:
# Check the number of unique values in each column
charity_df[charity_cat].nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
dtype: int64

In [6]:
# Check the unique value counts to see if binning is require
name_counts = charity_df.NAME.value_counts()
name_counts.head(50)

PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                        408
PTA TEXAS CONGRESS                                                     368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                          331
ALPHA PHI SIGMA                                                        313
TOASTMASTERS INTERNATIONAL                                             293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                      287
LITTLE LEAGUE BASEBALL INC                                             277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                               266
MOMS CLUB                                                              210
INTERNATIONAL ASSOCIATION

In [7]:
# Determine which values to replace
replace_name = list(name_counts[name_counts < 100].index)

# Replace in DataFrame
for name in replace_name:
    charity_df.NAME = charity_df.NAME.replace(name,"Other")


# Check to make sure binning was successful
charity_df.NAME.value_counts()

Other                                                                 25987
PARENT BOOSTER USA INC                                                 1260
TOPS CLUB INC                                                           765
UNITED STATES BOWLING CONGRESS INC                                      700
WASHINGTON STATE UNIVERSITY                                             492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                         408
PTA TEXAS CONGRESS                                                      368
SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC                           331
ALPHA PHI SIGMA                                                         313
TOASTMASTERS INTERNATIONAL                                              293
MOST WORSHIPFUL STRINGER FREE AND ACCEPTED MASONS                       287
LITTLE LEAGUE BASEBALL INC                                              277
INTERNATIONAL ASSOCIATION OF LIONS CLUBS                                266
MOMS CLUB   

In [8]:
# Check the unique value counts to see if binning is require
class_counts = charity_df.CLASSIFICATION.value_counts()
class_counts.head(50)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C4100        6
C1720        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C1246        2
C1256        2
C3200        2
C1267        2
C1234        2
C1728        1
C1245        1
C2190        1
C1580        1
C8210        1
Name: CLASSIFICATION, dtype: int64

In [9]:
# Determine which values to replace
replace_class = list(class_counts[class_counts < 120].index)

# Replace in DataFrame
for classs in replace_class:
    charity_df.CLASSIFICATION = charity_df.CLASSIFICATION.replace(classs,"Other")


# Check to make sure binning was successful
charity_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [10]:
# Check the unique value counts to see if binning is require
app_counts = charity_df.APPLICATION_TYPE.value_counts()
app_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [11]:

# Determine which values to replace
replace_app = list(app_counts[app_counts < 500].index)

# Replace in DataFrame
for app in replace_app:
    charity_df.APPLICATION_TYPE = charity_df.APPLICATION_TYPE.replace(app,"Other")


# Check to make sure binning was successful
charity_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [12]:
charity_df.head(50)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,Other,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,Other,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,Other,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,Other,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,Other,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
5,10556855,Other,T3,Independent,C1200,Preservation,Trust,1,0,N,5000,1
6,10558440,Other,T3,Independent,C1000,Preservation,Trust,1,100000-499999,N,31452,1
7,10566033,Other,T3,Independent,C2000,Preservation,Trust,1,10M-50M,N,7508025,1
8,10570430,Other,T7,Independent,C1000,ProductDev,Trust,1,1-9999,N,94389,1
9,10571689,Other,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [13]:
# Create the OneHotEncoder instance
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(sparse=False)

# Fit the encoder and produce encoded DataFrame
encode_df = pd.DataFrame(enc.fit_transform(charity_df.NAME.values.reshape(-1,1)))

# Rename encoded columns
encode_df.columns = enc.get_feature_names(['NAME'])
encode_df.head()

,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CIVITAN INTERNATIONAL,NAME_DEMOLAY INTERNATIONAL,NAME_FARMERS EDUCATIONAL AND COOPERATIVE UNION OF AMERICA,NAME_HABITAT FOR HUMANITY INTERNATIONAL INC,NAME_HONOR SOCIETY OF PHI KAPPA PHI,NAME_INTERNATIONAL ASSOCIATION OF LIONS CLUBS,NAME_INTERNATIONAL ASSOCIATION OF SHEET METAL AIR RAIL & TRANSPORTATION,...,NAME_SOROPTIMIST INTERNATIONAL OF THE AMERICAS INC,NAME_TENNESSEE ORDER OF THE EASTERN STAR,NAME_THE UNITED STATES PONY CLUBS INC,NAME_TOASTMASTERS INTERNATIONAL,NAME_TOPS CLUB INC,NAME_UNITED STATES BOWLING CONGRESS INC,NAME_UNIVERSITY OF WYOMING,NAME_VETERANS OF FOREIGN WARS OF THE UNITED STATES AUXILIARY,NAME_WASHINGTON STATE GRANGE,NAME_WASHINGTON STATE UNIVERSITY
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Fit the encoder and produce encoded DataFrame
encode2_df = pd.DataFrame(enc.fit_transform(charity_df.CLASSIFICATION.values.reshape(-1,1)))

# Rename encoded columns
encode2_df.columns = enc.get_feature_names(['CLASSIFICATION'])
encode2_df.head()

,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C4000,CLASSIFICATION_C7000,CLASSIFICATION_Other
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Fit the encoder and produce encoded DataFrame
encode3_df = pd.DataFrame(enc.fit_transform(charity_df.APPLICATION_TYPE.values.reshape(-1,1)))

# Rename encoded columns
encode3_df.columns = enc.get_feature_names(['APPLICATION_TYPE'])
encode3_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Check the unique value counts to see if binning is require
income_counts = charity_df.INCOME_AMT.value_counts()
income_counts

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [17]:
charity_df.loc[charity_df['INCOME_AMT'] == '0', 'income'] = '1'  
charity_df.loc[charity_df['INCOME_AMT'] == '100000-499999', 'income'] = '499999'  
charity_df.loc[charity_df['INCOME_AMT'] == '25000-99999', 'income'] = '99999'
charity_df.loc[charity_df['INCOME_AMT'] == '1M-5M', 'income'] = '4999999'
charity_df.loc[charity_df['INCOME_AMT'] == '1-9999', 'income'] = '9999'
charity_df.loc[charity_df['INCOME_AMT'] == '10000-24999', 'income'] = '24999'
charity_df.loc[charity_df['INCOME_AMT'] == '10M-50M', 'income'] = '49999999'
charity_df.loc[charity_df['INCOME_AMT'] == '5M-10M', 'income'] = '999999'
charity_df.loc[charity_df['INCOME_AMT'] == '50M+', 'income'] = '499999999'
charity_df

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,income
0,10520599,Other,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,1
1,10531628,Other,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,9999
2,10547893,Other,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,1
3,10553066,Other,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,24999
4,10556103,Other,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,499999
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,Other,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,1
34295,996010315,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,1
34296,996012607,Other,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,1
34297,996015768,Other,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,1


In [18]:
charity_df['income'] = charity_df['income'].astype(int)


In [22]:
# Merge the two DataFrames together and drop the Country column
charity2 = charity_df.merge(encode_df,left_index=True,right_index=True).drop("NAME",1)

In [23]:
# Merge the two DataFrames together and drop the Country column
charity3 = charity2.merge(encode2_df,left_index=True,right_index=True).drop("CLASSIFICATION",1)
charity3

,EIN,APPLICATION_TYPE,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,...,NAME_WASHINGTON STATE UNIVERSITY,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C1700,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C4000,CLASSIFICATION_C7000,CLASSIFICATION_Other
0,10520599,T10,Independent,ProductDev,Association,1,0,N,5000,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10531628,T3,Independent,Preservation,Co-operative,1,1-9999,N,108590,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,10547893,T5,CompanySponsored,ProductDev,Association,1,0,N,5000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,10553066,T3,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,10556103,T3,Independent,Heathcare,Trust,1,100000-499999,N,142590,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,T4,Independent,ProductDev,Association,1,0,N,5000,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34295,996010315,T4,CompanySponsored,ProductDev,Association,1,0,N,5000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
34296,996012607,T3,CompanySponsored,Preservation,Association,1,0,N,5000,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34297,996015768,T5,Independent,ProductDev,Association,1,0,N,5000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [24]:
# Merge the two DataFrames together and drop the Country column
charity4 = charity3.merge(encode3_df,left_index=True,right_index=True).drop("APPLICATION_TYPE",1)
charity4

,EIN,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,income,...,CLASSIFICATION_Other,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8
0,10520599,Independent,ProductDev,Association,1,0,N,5000,1,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10531628,Independent,Preservation,Co-operative,1,1-9999,N,108590,1,9999,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,10547893,CompanySponsored,ProductDev,Association,1,0,N,5000,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,10553066,CompanySponsored,Preservation,Trust,1,10000-24999,N,6692,1,24999,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,10556103,Independent,Heathcare,Trust,1,100000-499999,N,142590,1,499999,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,Independent,ProductDev,Association,1,0,N,5000,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34295,996010315,CompanySponsored,ProductDev,Association,1,0,N,5000,0,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34296,996012607,CompanySponsored,Preservation,Association,1,0,N,5000,0,1,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34297,996015768,Independent,ProductDev,Association,1,0,N,5000,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [25]:
# Merge the two DataFrames together and drop the Country column
charity5 = charity4.drop("INCOME_AMT",1)
charity5

,EIN,AFFILIATION,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,income,NAME_ALPHA PHI SIGMA,...,CLASSIFICATION_Other,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8
0,10520599,Independent,ProductDev,Association,1,N,5000,1,1,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10531628,Independent,Preservation,Co-operative,1,N,108590,1,9999,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,10547893,CompanySponsored,ProductDev,Association,1,N,5000,0,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,10553066,CompanySponsored,Preservation,Trust,1,N,6692,1,24999,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,10556103,Independent,Heathcare,Trust,1,N,142590,1,499999,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,Independent,ProductDev,Association,1,N,5000,0,1,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34295,996010315,CompanySponsored,ProductDev,Association,1,N,5000,0,1,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34296,996012607,CompanySponsored,Preservation,Association,1,N,5000,0,1,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34297,996015768,Independent,ProductDev,Association,1,N,5000,1,1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [27]:
# Fit the encoder and produce encoded DataFrame
encode4_df = pd.DataFrame(enc.fit_transform(charity_df.AFFILIATION.values.reshape(-1,1)))

# Rename encoded columns
encode4_df.columns = enc.get_feature_names(['AFFILIATION'])
encode4_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional
0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0


In [28]:
# Fit the encoder and produce encoded DataFrame
encode5_df = pd.DataFrame(enc.fit_transform(charity_df.USE_CASE.values.reshape(-1,1)))

# Rename encoded columns
encode5_df.columns = enc.get_feature_names(['USE_CASE'])
encode5_df.head()

,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev
0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,1.0,0.0
4,0.0,1.0,0.0,0.0,0.0


In [29]:
# Fit the encoder and produce encoded DataFrame
encode6_df = pd.DataFrame(enc.fit_transform(charity_df.ORGANIZATION.values.reshape(-1,1)))

# Rename encoded columns
encode6_df.columns = enc.get_feature_names(['ORGANIZATION'])
encode6_df.head()

,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


In [30]:
# Fit the encoder and produce encoded DataFrame
encode7_df = pd.DataFrame(enc.fit_transform(charity_df.SPECIAL_CONSIDERATIONS .values.reshape(-1,1)))

# Rename encoded columns
encode7_df.columns = enc.get_feature_names(['SPECIAL_CONSIDERATIONS'])
encode7_df.head()

,SPECIAL_CONSIDERATIONS _N,SPECIAL_CONSIDERATIONS _Y
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0


In [36]:
# Merge the two DataFrames together and drop the Country column
charity6 = charity5.merge(encode4_df,left_index=True,right_index=True)
charity6

,EIN,USE_CASE,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,income,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,...,AFFILIATION_Independent_x,AFFILIATION_National_x,AFFILIATION_Other_x,AFFILIATION_Regional_x,AFFILIATION_CompanySponsored_y,AFFILIATION_Family/Parent_y,AFFILIATION_Independent_y,AFFILIATION_National_y,AFFILIATION_Other_y,AFFILIATION_Regional_y
0,10520599,ProductDev,Association,1,N,5000,1,1,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,10531628,Preservation,Co-operative,1,N,108590,1,9999,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,10547893,ProductDev,Association,1,N,5000,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,10553066,Preservation,Trust,1,N,6692,1,24999,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,10556103,Heathcare,Trust,1,N,142590,1,499999,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,ProductDev,Association,1,N,5000,0,1,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
34295,996010315,ProductDev,Association,1,N,5000,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34296,996012607,Preservation,Association,1,N,5000,0,1,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
34297,996015768,ProductDev,Association,1,N,5000,1,1,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [39]:
# Merge the two DataFrames together and drop the Country column
charity7 = charity6.merge(encode5_df,left_index=True,right_index=True).drop("USE_CASE",1)
charity7

,EIN,ORGANIZATION,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,income,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,...,AFFILIATION_Family/Parent_y,AFFILIATION_Independent_y,AFFILIATION_National_y,AFFILIATION_Other_y,AFFILIATION_Regional_y,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev
0,10520599,Association,1,N,5000,1,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,10531628,Co-operative,1,N,108590,1,9999,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,10547893,Association,1,N,5000,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,10553066,Trust,1,N,6692,1,24999,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,10556103,Trust,1,N,142590,1,499999,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,Association,1,N,5000,0,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
34295,996010315,Association,1,N,5000,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
34296,996012607,Association,1,N,5000,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
34297,996015768,Association,1,N,5000,1,1,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [40]:
# Merge the two DataFrames together and drop the Country column
charity8 = charity7.merge(encode6_df,left_index=True,right_index=True).drop("ORGANIZATION",1)
charity8

,EIN,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,income,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CIVITAN INTERNATIONAL,...,AFFILIATION_Regional_y,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,10520599,1,N,5000,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,10531628,1,N,108590,1,9999,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,10547893,1,N,5000,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,10553066,1,N,6692,1,24999,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,10556103,1,N,142590,1,499999,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,1,N,5000,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
34295,996010315,1,N,5000,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
34296,996012607,1,N,5000,0,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
34297,996015768,1,N,5000,1,1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [41]:
# Merge the two DataFrames together and drop the Country column
charity_cleaned = charity8.merge(encode7_df,left_index=True,right_index=True).drop("SPECIAL_CONSIDERATIONS",1)
charity_cleaned

,EIN,STATUS,ASK_AMT,IS_SUCCESSFUL,income,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CIVITAN INTERNATIONAL,NAME_DEMOLAY INTERNATIONAL,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS _N,SPECIAL_CONSIDERATIONS _Y
0,10520599,1,5000,1,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
1,10531628,1,108590,1,9999,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,10547893,1,5000,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
3,10553066,1,6692,1,24999,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,10556103,1,142590,1,499999,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,1,5000,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
34295,996010315,1,5000,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
34296,996012607,1,5000,0,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
34297,996015768,1,5000,1,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [42]:
charity_cleaned.dtypes

EIN                            int64
STATUS                         int64
ASK_AMT                        int64
IS_SUCCESSFUL                  int64
income                         int64
                              ...   
ORGANIZATION_Co-operative    float64
ORGANIZATION_Corporation     float64
ORGANIZATION_Trust           float64
SPECIAL_CONSIDERATIONS _N    float64
SPECIAL_CONSIDERATIONS _Y    float64
Length: 77, dtype: object

In [43]:
# Fit the encoder and produce encoded DataFrame
encode8_df = pd.DataFrame(enc.fit_transform(charity_df.IS_SUCCESSFUL .values.reshape(-1,1)))

# Rename encoded columns
encode8_df.columns = enc.get_feature_names(['IS_SUCCESSFUL'])
encode8_df.head()

,IS_SUCCESSFUL_0,IS_SUCCESSFUL_1
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,0.0,1.0
4,0.0,1.0


In [54]:
# Merge the two DataFrames together and drop the Country column
attrition_df = charity_cleaned.merge(encode8_df,left_index=True,right_index=True).drop("IS_SUCCESSFUL",1)
attrition_df

,EIN,STATUS,ASK_AMT,income,NAME_ALPHA PHI SIGMA,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES INC,NAME_AMERICAN ASSOCIATION OF UNIVERSITY WOMEN,NAME_CIVITAN INTERNATIONAL,NAME_DEMOLAY INTERNATIONAL,NAME_FARMERS EDUCATIONAL AND COOPERATIVE UNION OF AMERICA,...,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS _N,SPECIAL_CONSIDERATIONS _Y,IS_SUCCESSFUL_0,IS_SUCCESSFUL_1
0,10520599,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,10531628,1,108590,9999,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,10547893,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,10553066,1,6692,24999,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
4,10556103,1,142590,499999,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,996009318,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
34295,996010315,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
34296,996012607,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
34297,996015768,1,5000,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [55]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["IS_SUCCESSFUL_1"].values
X = attrition_df.drop(["IS_SUCCESSFUL_0","IS_SUCCESSFUL_1"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [56]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [57]:
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 616       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 6         
Total params: 667
Trainable params: 667
Non-trainable params: 0
_________________________________________________________________


In [58]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [59]:
# Train the model
fit_model = nn.fit(X_train,y_train,epochs=100)

Train on 25724 samples
Epoch 1/100
25724/25724 [==============================] - 2s 82us/sample - loss: 164035.3902 - accuracy: 0.5042
Epoch 2/100
25724/25724 [==============================] - 1s 51us/sample - loss: 107298.0229 - accuracy: 0.4996
Epoch 3/100
25724/25724 [==============================] - 1s 51us/sample - loss: 106758.9343 - accuracy: 0.5029
Epoch 4/100
25724/25724 [==============================] - 1s 50us/sample - loss: 90621.5806 - accuracy: 0.5020
Epoch 5/100
25724/25724 [==============================] - 1s 50us/sample - loss: 99042.8673 - accuracy: 0.5064
Epoch 6/100
25724/25724 [==============================] - 1s 52us/sample - loss: 94128.3614 - accuracy: 0.4982
Epoch 7/100
25724/25724 [==============================] - 1s 51us/sample - loss: 73737.9746 - accuracy: 0.5012
Epoch 8/100
25724/25724 [==============================] - 1s 51us/sample - loss: 75361.7872 - accuracy: 0.4994
Epoch 9/100
25724/25724 [==============================] - 1s 51us/sample - lo

25724/25724 [==============================] - 1s 53us/sample - loss: 3499.4348 - accuracy: 0.5050
Epoch 74/100
25724/25724 [==============================] - 1s 52us/sample - loss: 2035.3722 - accuracy: 0.4983
Epoch 75/100
25724/25724 [==============================] - 1s 52us/sample - loss: 3244.9522 - accuracy: 0.5059
Epoch 76/100
25724/25724 [==============================] - 1s 52us/sample - loss: 3979.4597 - accuracy: 0.4997
Epoch 77/100
25724/25724 [==============================] - 1s 50us/sample - loss: 5093.0441 - accuracy: 0.5043
Epoch 78/100
25724/25724 [==============================] - 1s 50us/sample - loss: 1288.8485 - accuracy: 0.5010
Epoch 79/100
25724/25724 [==============================] - 1s 50us/sample - loss: 5962.8853 - accuracy: 0.5042
Epoch 80/100
25724/25724 [==============================] - 1s 51us/sample - loss: 1368.0043 - accuracy: 0.5068
Epoch 81/100
25724/25724 [==============================] - 1s 51us/sample - loss: 2689.8632 - accuracy: 0.4977
Epoch

In [60]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/1 - 0s - loss: 185.8356 - accuracy: 0.5332
Loss: 370.9821101063264, Accuracy: 0.5331778526306152
